# Interactive plot with a slider

## Setup and data source

Our data source are stats of the first two Pokémon Generations!

In [ ]:
### import libraries
import numpy as np
import pandas as pd
from bokeh.models import ColumnDataSource

### read in data and construct column data source
df = pd.read_csv('data/pokemon_250.csv')

datasource = ColumnDataSource(data={
    "name" : df["Name"],
    "main_type" : df["Type 1"],
    "overall" : df["Total"],
    "health" : df["HP"],
    "attack" : df["Attack"],
    "defense" : df["Defense"],
    "speed" : df["Speed"]
})

## Basic code for bokeh plot

In [ ]:
### import libraries
from bokeh.plotting import figure
from bokeh.models import HoverTool

### create canvas
plot = figure(title="Plot A - Pokemon HP Slider", x_axis_label ="Attack", y_axis_label ="Defense",
                plot_width=600, plot_height=600)

### draw data points and add a tooltip
plot.circle("attack", "defense", size=10, source=datasource)

tooltips = [("Name","@name"),("Health","@health")]
plot.add_tools(HoverTool(tooltips=tooltips))

## Build bokeh app

We want to add a slider, with which the user can define lower and upper boundaries for Pokémon health stats

In [38]:
### import libraries
from bokeh.io import curdoc
from bokeh.layouts import row, column
from bokeh.models.widgets import RangeSlider

### add a slider widget, to define a threshold for different health values
### initially values are 10(min) and 255(max)
hp_slider = RangeSlider(start = 10, end = 255, step = 1, value = (10,255), title = 'Pokemon Health')


### define update function, which changes the data source
### "attr, old, new" is always required in the update function!
def update_hp(attr, old, new):
    
    ### define lower and upper boundary
    lower_boundary_hp = hp_slider.value[0]
    upper_boundary_hp = hp_slider.value[1]
    
    ### source data from a new dataframe, in which we only have Pokémon with health within the defined boundaries
    new_df = df.loc[(df["HP"] > lower_boundary_hp) & (df["HP"] < upper_boundary_hp)]

    datasource.data = dict(
        name = new_df["Name"],
        main_type = new_df["Type 1"],
        overall = new_df["Total"],
        health = new_df["HP"],
        attack = new_df["Attack"],
        defense = new_df["Defense"],
        speed = new_df["Speed"]
    )

### when user gives input to slider, execute updating function
hp_slider.on_change('value', update_hp)

### add widget and plot to layout
layout = row(column(hp_slider), plot)
curdoc().add_root(layout)

### in order to show app in your browser, use command line and type "bokeh serve your_app_name.ipynb --show"